## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-07-21 10:58:44,196 root         INFO     Running on pakhi


In [2]:
from ceciestunepipe.file import bcistructure as et

### Get the file locations for a session (day) of recordings

In [122]:
reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': '2021-07-25'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [123]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-25',
 'alsa': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-25/alsa',
 'kwik': '/scratch/earneodo/s_b1253_21/alsa/kwik/2021-07-25',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-07-25/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-25/alsa',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1253_21/alsa/msort/2021-07-25',
 'ksort': '/scratch/earneodo/s_b1253_21/alsa/ksort/2021-07-25'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [124]:
from ceciestunepipe.util.sound import boutsearch as bs

In [126]:
### load the search/spectrogram parameters
### these functions are necessary to load the pickle files, which is bad.
### need to not write modules/functions in the pickle

# function for getting one channel out of a wave file
def read_wav_chan(wav_path: str, chan_id: int=0) -> tuple:
    s_f, x = wavfile.read(wav_path, mmap=True)
    return s_f, x[:, chan_id]

def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n

hparams_file_path = os.path.join(bouts_folder, 'bout_search_params.pickle')
with open(hparams_file_path, 'rb') as fh:
    hparams = pickle.load(fh)

In [127]:
#bpd, x, p = bs.get_bouts_in_file(one_wav_path, hparams)

bouts_auto_file_path = os.path.join(bouts_folder, hparams['bout_auto_file'])

#load. It is important to reset index because the manual curation requires unique indexing
bpd = pd.read_pickle(bouts_auto_file_path).reset_index(drop=True)
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform
0,270015,275560,12960720,13226880,"[51.84148182872022, 15.918929262271554, 27.263...",1.718223,111.134623,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5545,"[[270015, 270090], [270910, 270955], [271205, ...",7,"[7, 14, 43, 65, 92, 146, 185, 230, 238, 247, 2...",23,241.086957,"[-1, 2, 2, 1, 1, -1, 1, 3, 4, 4, 3, 3, 2, 4, 3..."


###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [128]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [129]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)

cleanup(bpd)

#### compute the spectrograms

In [130]:
bpd['spectrogram'] = bpd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + bouts_auto_file_path)
bpd.to_pickle(bouts_auto_file_path)

2021-07-26 08:55:35,310 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-25/alsa/bouts_ceciestunepipe/bout_auto.pickle


In [131]:
bpd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,270015,275560,12960720,13226880,"[51.84148182872022, 15.918929262271554, 27.263...",1.718223,111.134623,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5545,"[[270015, 270090], [270910, 270955], [271205, ...",7,"[7, 14, 43, 65, 92, 146, 185, 230, 238, 247, 2...",23,241.086957,"[-1, 2, 2, 1, 1, -1, 1, 3, 4, 4, 3, 3, 2, 4, 3...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1206105,1212095,57893040,58180560,"[9.130886719114956, 12.788625242666747, 9.2806...",1.718223,61.955870,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5990,"[[1206105, 1206135], [1207105, 1207145], [1207...",10,"[1, 13, 50, 57, 187, 201, 231, 241, 250, 257, ...",42,142.619048,"[-5, -4, -3, -3, -5, -5, -5, -4, -3, -4, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,461930,470175,22172640,22568400,"[46.54651527639073, 34.89179574950654, 29.2582...",1.707197,121.737039,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,8245,"[[461930, 461975], [462260, 462315], [463010, ...",15,"[6, 19, 66, 104, 128, 144, 156, 216, 259, 279,...",34,242.500000,"[-3, -8, -10, -13, -12, -9, -6, -1, 1, -1, -3,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,827680,833470,39728640,40006560,"[32.86802508710033, 8.378392973074682, 3.06614...",1.707197,81.016835,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5790,"[[827680, 827710], [828195, 828230], [828495, ...",10,"[6, 49, 103, 110, 163, 231, 243, 296, 305, 321...",27,214.444444,"[3, 3, 3, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 3, 3, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,1645,53875,78960,2586000,"[1.7030605399977832, 16.283014894590835, 19.62...",5.158895,122.887253,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,52230,"[[1645, 1715], [3355, 3430], [5025, 5095], [52...",186,"[6, 276, 348, 613, 681, 721, 746, 753, 907, 94...",500,104.460000,"[2, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 3, 2, 2, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,104055,146560,4994640,7034880,"[2.594915558576812, 3.7965567700578453, 2.4429...",5.158895,140.527588,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,42505,"[[104055, 104145], [104170, 104210], [105640, ...",150,"[1, 13, 28, 323, 332, 346, 362, 694, 704, 717,...",356,119.396067,"[2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,1176290,1231455,56461920,59109840,"[1.638526596994977, 2.201699340975124, 4.10595...",5.158895,166.158944,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,55165,"[[1176290, 1176380], [1177360, 1177700], [1178...",204,"[5, 14, 215, 223, 230, 240, 247, 255, 263, 277...",718,76.831476,"[4, 5, 5, 5, 6, 6, 6, 5, 4, 5, 6, 5, 5, 5, 5, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,1125350,1166505,54016800,55992240,"[4.733404919668049, 19.520180162262026, 40.364...",2.983902,143.486453,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,41155,"[[1125350, 1125380], [1126635, 1126720], [1126...",157,"[3, 234, 260, 268, 304, 529, 577, 603, 646, 65...",400,102.887500,"[-1, -2, -2, -3, -2, -1, -3, -2, -2, -1, -2, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,1413255,1432745,67836240,68771760,"[1.3041575487678196, 2.298452195735337, 1.8964...",2.983902,106.937015,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,19490,"[[1413255, 1413335], [1415125, 1415205], [1416...",52,"[4, 12, 386, 395, 408, 437, 477, 572, 689, 764...",129,151.085271,"[-4, -3, -4, -4, -3, -4, -4, -4, -4, -3, -3, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9,1454500,1500040,69816000,72001920,"[10.939668634044576, 21.553817133919626, 40.54...",2.983902,143.011223,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,45540,"[[1454500, 1454575], [1455885, 1455960], [1457.

## inspect the bouts and curate them

#### visualize one bout

In [132]:
bpd.iloc[0]

start_ms                                                     270015
end_ms                                                       275560
start_sample                                               12960720
end_sample                                                 13226880
p_step            [51.84148182872022, 15.918929262271554, 27.263...
rms_p                                                      1.718223
peak_p                                                   111.134623
bout_check                                                     True
file              /mnt/sphere/speech_bci/raw_data/s_b1253_21/202...
len_ms                                                         5545
syl_in            [[270015, 270090], [270910, 270955], [271205, ...
n_syl                                                             7
peaks_p           [7, 14, 43, 65, 92, 146, 185, 230, 238, 247, 2...
n_peaks                                                          23
l_p_ratio                                       

In [133]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [134]:
def viz_one_bout(df: pd.Series, sub_sample=10):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bpd.iloc[0])

    'data': [{'type': 'scatter',
              'uid': '6314ed60-994d-493a-b783-6…

In [135]:
bpd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,270015,275560,12960720,13226880,"[51.84148182872022, 15.918929262271554, 27.263...",1.718223,111.134623,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5545,"[[270015, 270090], [270910, 270955], [271205, ...",7,"[7, 14, 43, 65, 92, 146, 185, 230, 238, 247, 2...",23,241.086957,"[-1, 2, 2, 1, 1, -1, 1, 3, 4, 4, 3, 3, 2, 4, 3...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1206105,1212095,57893040,58180560,"[9.130886719114956, 12.788625242666747, 9.2806...",1.718223,61.955870,True,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5990,"[[1206105, 1206135], [1207105, 1207145], [1207...",10,"[1, 13, 50, 57, 187, 201, 231, 241, 250, 257, ...",42,142.619048,"[-5, -4, -3, -3, -5, -5, -5, -4, -3, -4, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [136]:
bpd.reset_index(drop=True, inplace=True)

In [137]:
## Set confusing by default, will only be False once asserted bout/or not
bpd['confusing'] = True
bpd['bout_check'] = False

In [138]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [139]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [118]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

45

### save it

In [119]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [120]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2021-07-25 17:49:07,034 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-24/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [121]:
viz_bout.bouts_pd.head(5)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,622285,627315,29869680,30111120,"[25.934583301732737, 28.301503100312782, 16.85...",1.870506,37.432538,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5030,"[[622285, 622325], [622745, 622800], [623815, ...",6,"[1, 41, 93, 307, 317, 476, 486, 569, 623, 643,...",15,335.333333,"[-7, -8, -7, -7, -7, -8, -8, -8, -7, -7, -9, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
1,878515,886980,42168720,42575040,"[35.9270888229864, 32.73564202449041, 22.82004...",1.870506,64.426111,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,8465,"[[878515, 878550], [878630, 878660], [879155, ...",11,"[7, 15, 23, 87, 128, 148, 198, 209, 223, 230, ...",41,206.463415,"[18, 77, 2, -3, 15, -18, 31, 21, -10, 32, 21, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
2,650930,656630,31244640,31518240,"[51.47069301801644, 47.94933382274568, 36.8882...",1.665845,55.325448,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,5700,"[[650930, 650975], [652420, 652455], [652820, ...",7,"[65, 111, 155, 162, 228, 266, 275, 298, 308, 3...",22,259.090909,"[-10, -10, -9, -10, -11, -10, -10, -11, -10, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
3,436400,442655,20947200,21247440,"[0.5029896761807137, 1.9081208310755633, 9.543...",1.193061,63.051033,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,6255,"[[436400, 436445], [436615, 436655], [436865, ...",13,"[3, 13, 25, 36, 44, 53, 94, 314, 450, 464, 474...",46,135.978261,"[2, 1, 1, 1, 2, 1, -1, 0, 2, 1, -1, 0, 1, 0, 0...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
4,859740,865745,41267520,41555760,"[8.78734644905731, 5.896271887054869, 5.934815...",1.134268,92.908850,False,/mnt/sphere/speech_bci/raw_data/s_b1253_21/202...,6005,"[[859740, 859775], [860475, 860510], [860635, ...",8,"[2, 11, 71, 81, 148, 179, 282, 586, 598, 605, ...",25,240.200000,"[0, -1, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",False
